In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
from pathlib import Path

from IPython.display import display
from HMM_target_detector_20251204 import HMM_target_detector

In [3]:
ANGLE_RANGE = 2*np.pi
CODE_COLOR_MAP = {0 : 'yellow',
                  1 : 'blue',
                  2 : 'purple'}

In [4]:
prob_code0_in_states = np.array([1, 0, 1, 1])
prob_code1_in_states = np.array([0, 1, 0, 0])
EMISSION_MATRIX_TARGET = np.vstack([prob_code0_in_states, prob_code1_in_states])

prob_code0_in_states = np.array([1, 1, 1, 1])
prob_code1_in_states = np.array([0, 0, 0, 0])
EMISSION_MATRIX_NONTARGET = np.vstack([prob_code0_in_states, prob_code1_in_states])

EMISSION_MATRIX_TARGETS = np.stack([EMISSION_MATRIX_TARGET, EMISSION_MATRIX_NONTARGET])
EMISSION_MATRIX_TARGETS, EMISSION_MATRIX_TARGETS.shape

(array([[[1, 0, 1, 1],
         [0, 1, 0, 0]],
 
        [[1, 1, 1, 1],
         [0, 0, 0, 0]]]),
 (2, 2, 4))

In [5]:
agent = HMM_target_detector(emission_matrix_targets=EMISSION_MATRIX_TARGETS)

In [6]:
initial_angle_slider = widgets.IntSlider(
    value=0,  
    min=0,
    max=360,
    description="Initial Angle",
    style={'description_width': 'initial'},
    continuous_update=True,
    layout=widgets.Layout(width="600px"),
)

interactive_plot = widgets.interactive(
    agent.observe_angle_state_pair, initial_angle=initial_angle_slider)
display(interactive_plot)

interactive(children=(IntSlider(value=0, description='Initial Angle', layout=Layout(width='600px'), max=360, s…

In [7]:
save_folder = Path(f'../test_trials__detection_model_20251204_150300_four_states')
save_folder.mkdir(parents=True, exist_ok=True)

In [8]:
across_trial_params = dict()
across_trial_params['trial_num'] = 0
across_trial_params['max_iter'] = 100
across_trial_params['num_trials'] = 10

across_trial_arrays = dict()
across_trial_arrays['posterior_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter']+1, agent.num_beliefs))
across_trial_arrays['expected_forecast_t_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter'], agent.candidate_steps.shape[0]))
across_trial_arrays['decision_type_across_trials'] = np.empty((across_trial_params['num_trials'], across_trial_params['max_iter']), dtype=np.dtypes.StringDType())
across_trial_arrays['angles_visited_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter']))
across_trial_arrays['steps_taken_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter']))
across_trial_arrays['entropy_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter']))
across_trial_arrays['time_taken_per_trial'] = np.zeros(across_trial_params['num_trials'])
across_trial_arrays['codes_received_across_trials'] = np.zeros((across_trial_params['num_trials'], across_trial_params['max_iter']))

emission_matrix_true = EMISSION_MATRIX_TARGETS[0]

while across_trial_params['trial_num'] < across_trial_params['num_trials']:

    agent = HMM_target_detector(emission_matrix_targets=EMISSION_MATRIX_TARGETS)
    while agent.within_trial_params["time_ind"]<agent.within_trial_params['max_iter'] and agent.within_trial_params['stopping_num'] < 2:
        if np.isclose(agent.within_trial_arrays['posterior_t'][agent.within_trial_params["time_ind"]-1], 1, atol=1e-2).any() or agent.within_trial_params['stopping_num']>0:
            agent.within_trial_params['stopping_num']+=1

        agent.move_and_sample_from_object()

        agent.update_posterior_and_compute_current_entropy()

        agent.debug_within_trial_print_statements()

        agent.forecast_and_compute_expected_entropy_from_actions()

        agent.compute_expected_entropy_change_from_each_action()
        
        agent.determine_decision_from_forecasted_entropy_change()
        agent.within_trial_params["time_ind"]+=1
    
    summary_df = agent.get_summary_of_trial()
    summary_df.to_csv(f'{save_folder}/trial_{across_trial_params["trial_num"]}.csv')

    across_trial_arrays['posterior_across_trials'][across_trial_params['trial_num'],:agent.within_trial_params["time_ind"]+1,:] = np.vstack([agent.prior_target_prob, agent.within_trial_arrays['posterior_t'][:agent.within_trial_params["time_ind"]]])
    across_trial_arrays['entropy_across_trials'][across_trial_params['trial_num'], :agent.within_trial_params["time_ind"]] = agent.within_trial_arrays['current_entropyS_t'][:agent.within_trial_params["time_ind"]]
    across_trial_arrays['expected_forecast_t_across_trials'][across_trial_params['trial_num'], :agent.within_trial_params["time_ind"],:] = agent.within_trial_arrays['expected_forecast_t_S_per_step'][:agent.within_trial_params["time_ind"]]
    across_trial_arrays['steps_taken_across_trials'][across_trial_params['trial_num'],          :] = agent.within_trial_arrays['steps_taken']
    across_trial_arrays['angles_visited_across_trials'][across_trial_params['trial_num'],       :] = agent.within_trial_arrays['angles_visited']
    across_trial_arrays['codes_received_across_trials'][across_trial_params['trial_num'],       :] = agent.within_trial_arrays['code_received_t']
    across_trial_arrays['time_taken_per_trial'][across_trial_params['trial_num']] = agent.within_trial_params["time_ind"]
    across_trial_arrays['decision_type_across_trials'][across_trial_params['trial_num']] = agent.within_trial_arrays["decision_type_t"]
    across_trial_params['trial_num']+=1

Current angle: 0.0
Code received: 0.0
[0.375 0.5  ]
P(O1=[0.],S1|T1)=[0.25 0.   0.25 0.25], P(O1=[0.]|T1)=0.75, P(S1|O1=[0.],T1)=[0.33 0.   0.33 0.33], P(T|O1=[0.])=[0.42857143 0.57142857], H1=0.99
P(O1=[0.],S1|T2)=[0.25 0.25 0.25 0.25], P(O1=[0.]|T2)=1.0, P(S1|O1=[0.],T2)=[0.25 0.25 0.25 0.25], P(T|O1=[0.])=[0.42857143 0.57142857], H1=0.99
P(O2=0,O1=[0.]|Δθ2,T)=[0.5 1. ]
P(O2=0|O1=[0.],Δθ2,T)=[0.66666667 1.        ]
P(O2=1,O1=[0.]|Δθ2,T)=[0.25 0.  ]
P(O2=1|O1=[0.],Δθ2,T)=[0.33333333 0.        ]
S2 = 0.857x0.918 + 0.143x0.000 = 0.7871107149038481
P(O2=0,O1=[0.]|Δθ2,T)=[0.75 1.  ]
P(O2=0|O1=[0.],Δθ2,T)=[1. 1.]
P(O2=1,O1=[0.]|Δθ2,T)=[0. 0.]
P(O2=1|O1=[0.],Δθ2,T)=[0. 0.]
S2 = 1.000x0.985 + 0.000x0.000 = 0.9852281360342515
P(O2=0,O1=[0.]|Δθ2,T)=[0.5 1. ]
P(O2=0|O1=[0.],Δθ2,T)=[0.66666667 1.        ]
P(O2=1,O1=[0.]|Δθ2,T)=[0.25 0.  ]
P(O2=1|O1=[0.],Δθ2,T)=[0.33333333 0.        ]
S2 = 0.857x0.918 + 0.143x0.000 = 0.7871107149038481
S1=0.9852281360342515
S2=[0.78711071 0.98522814 0.78711071]
Δ

In [9]:
test_df = pd.read_csv(f'{save_folder}/trial_0.csv', index_col=0)
test_df

,posterior_t_T1,posterior_t_T2,expected_forecast_t_S_per_step_CW,expected_forecast_t_S_per_step_STAY,expected_forecast_t_S_per_step_CCW,angles_visited,steps_taken,current_entropyS_t,code_received_t,decision_type_t,time_taken
0,0.428571,0.571429,0.787111,0.985228,0.787111,0.0,1.570796,0.985228,0.0,random,4.0
1,1.000000,0.000000,0.000000,0.000000,0.000000,90.0,0.000000,0.000000,1.0,end,4.0
2,1.000000,0.000000,0.000000,0.000000,0.000000,90.0,0.000000,0.000000,1.0,end,4.0
3,1.000000,0.000000,0.000000,0.000000,0.000000,90.0,0.000000,0.000000,1.0,end,4.0


In [ ]:
num_trials = len(list(save_folder.glob('*.csv')))
trial_selection_limit = min(10, num_trials)
trial_df_cache = {}

def get_summary_df(trial_idx):
    capped_idx = min(trial_idx, num_trials - 1)
    if capped_idx not in trial_df_cache:
        trial_df_cache[capped_idx] = pd.read_csv(f"{save_folder}/trial_{capped_idx}.csv", index_col=0)
    return trial_df_cache[capped_idx]

def update(time_ind, trial_num):
    summary_df_for_trial = get_summary_df(trial_num)
    fig = plt.figure(figsize=(12, 6))
    gs = gridspec.GridSpec(1, 2, width_ratios=[4, 4], wspace=0.3, figure=fig)
    gs_signals = gridspec.GridSpecFromSubplotSpec(
        2, 1, height_ratios=[2,1], subplot_spec=gs[0, 1], hspace=0.3)

    ax_grid = fig.add_subplot(gs[0, 0])
    ax_diffusion = fig.add_subplot(gs_signals[0])
    ax_entropy = fig.add_subplot(gs_signals[1])

    current_angle = (summary_df_for_trial['angles_visited'][time_ind]%360)
    code_received = summary_df_for_trial['code_received_t'][time_ind]
    initial_radians = np.radians(current_angle)

    setup_details = {'title': 'Target', 'grid_extent': 10, 
                        'inner_radius' : 1, 'outer_radius' : 5,
                        'agent_radians' : initial_radians}
    agent.plot_code_dependent_regions(ax_grid, EMISSION_MATRIX_TARGETS[0], setup_details)
    agent.plot_agent_in_env(ax_grid, setup_details, code_received)

    if time_ind>0:
        prev_angle = (summary_df_for_trial['angles_visited'][time_ind-1]%360)
        initial_radians = np.radians(prev_angle)
        ax_grid.scatter(1.2*setup_details['outer_radius']*np.cos(initial_radians), 1.2*setup_details['outer_radius']*np.sin(initial_radians),
                s=200, marker='.', facecolor='r', edgecolor='k', alpha=0.6, zorder=4)
        
    if time_ind>1:
        prev_angle = (summary_df_for_trial['angles_visited'][time_ind-2]%360)
        initial_radians = np.radians(prev_angle)
        ax_grid.scatter(1.4*setup_details['outer_radius']*np.cos(initial_radians), 1.4*setup_details['outer_radius']*np.sin(initial_radians),
                s=200, marker='.', facecolor='r', edgecolor='k', alpha=0.3, zorder=4)

    for i in range(agent.num_beliefs):
        if i == 0:
            class_name = 'target'
        else:
            class_name = 'non-target'
        ax_diffusion.plot(summary_df_for_trial[f'posterior_t_T{int(i+1)}'][:int(time_ind+1)], marker='.', label=f'Class {class_name}')
    ax_diffusion.axhline(y=1.0, linestyle='dashed', color='k')
    ax_diffusion.set_xlim(0, summary_df_for_trial['time_taken'][0]+1)
    ax_diffusion.set_ylim(0.0, 1.05)
    ax_diffusion.set_xticks(np.arange(summary_df_for_trial['time_taken'][0]+2).astype(int))
    ax_diffusion.set_ylabel('Class probability')
    ax_diffusion.set_xlabel('Time index (k)')

    x_vals = np.arange(max(0, time_ind-2), min(summary_df_for_trial.shape[0], time_ind+3)).astype(int)
    ax_entropy.plot(x_vals, summary_df_for_trial['expected_forecast_t_S_per_step_CW'][max(0, time_ind-2):min(summary_df_for_trial.shape[0], time_ind+3)], marker='.')
    ax_entropy.plot(x_vals, summary_df_for_trial['expected_forecast_t_S_per_step_STAY'][max(0, time_ind-2):min(summary_df_for_trial.shape[0], time_ind+3)], marker='.')
    ax_entropy.plot(x_vals, summary_df_for_trial['expected_forecast_t_S_per_step_CCW'][max(0, time_ind-2):min(summary_df_for_trial.shape[0], time_ind+3)], marker='.')
    ax_entropy.axvline(x=time_ind, linestyle='dashed', color='k', label=f"Decision type:{summary_df_for_trial['decision_type_t'][time_ind]}")
    xlabels = np.arange(time_ind-2, min(summary_df_for_trial.shape[0], time_ind+3)).astype(int)
    ax_entropy.set_xticks(xlabels)
    ax_entropy.set_ylabel('Forecasted entropy')
    ax_entropy.legend(loc='lower left')
    ax_entropy.set_xlabel('Time index (k)')


trial_selector = widgets.ToggleButtons(
    options=[(f'Trial {i}', i) for i in range(trial_selection_limit)],
    description='Trial',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

time_ind_slider = widgets.IntSlider(
    value=0, min=0, max=max(0, get_summary_df(0).shape[0]-1), step=1,
    description="Time (k)",
    continuous_update=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="400px")
)


def _sync_slider_with_trial():
    df = get_summary_df(trial_selector.value)
    new_max = max(0, df.shape[0]-1)
    time_ind_slider.max = new_max
    if time_ind_slider.value > new_max:
        time_ind_slider.value = new_max


def _on_trial_change(change):
    if change['name'] == 'value':
        _sync_slider_with_trial()


trial_selector.observe(_on_trial_change, names='value')
_sync_slider_with_trial()

controls = widgets.HBox([trial_selector, time_ind_slider])
interactive_plot = widgets.interactive_output(
    update, {'time_ind': time_ind_slider, 'trial_num': trial_selector})
display(widgets.VBox([controls, interactive_plot]))

